# Goals of this script

1. Import the rMATS data
2. Apply a series of cutoffs
3. Randomly select a subset of the chosen events and obtain IGV screenshots or rMATS2sashimiplot (this is better and doesn't require manual load)
4. Then go through visually and score the events

# Cutoffs to use
1. FDR
2. dPSI
3. Junction Reads
4. Gene Expression - Use cuffdiff
5. No intervening exons

In [38]:
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None  # default='warn'
pd.options.mode.chained_assignment = None  # default='warn'
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib_venn as mplv
import matplotlib.ticker as ticker
import os
from math import *
from subprocess import *
import pybedtools as pbt
from glob import glob
import seaborn as sns
import statsmodels.formula.api as sm
import csv
import re
import scipy.stats as stats
import copy
csv.register_dialect("textdialect",delimiter='\t')
%matplotlib inline
mpl.rc("lines", markeredgewidth=0.5)

In [141]:
#Master Function
def master_rmats_validation(rmats_parsing_file,cuffdiff_file,df_name,bam1_list,bam2_list,label1,label2,exon_scale,intron_scale,sample_frac,sample_seed,flip_needed,fdr,dpsi,reads,one_variable_only,run_sashimi):
    
    test_df = pd.read_table(rmats_parsing_file)
    
    print(len(test_df),'number_of_SE_events')
    
    def test_filter(df,reads,fdr,dpsi,flip_needed):
        df = df[df.SAMPLE_1_mean_total > reads]
        df = df[df.SAMPLE_2_mean_total > reads]
        #now need to generate variable ID and filter based on highest junction read
        df['v_id'] = df.splice_id.str.split(':').str[1] + ':' + df.splice_id.str.split(':').str[2] + ':' + df.splice_id.str.split(':').str[5] + ":" + df.splice_id.str.split(':').str[6] + ":" + df.GeneID
        #make a combo_junctions column that adds up the junctions
        df['combo_junctions'] = df.SAMPLE_1_mean_total + df. SAMPLE_2_mean_total
        #sort based on these junctions with the highest first
        df.sort_values(by='combo_junctions',ascending=False,inplace=True)
        #drop duplicates keeping the highest based on variable ID
        df.drop_duplicates(subset='v_id',keep='first',inplace=True)
        #fdr filter
        df = df[df.FDR <= fdr]
        #dpsi filter
        df = df[df.IncLevelDifference.abs() >= dpsi]
        #the flip part is yes if you need to flip the deltaPSI to make positive deltaPSI = promote skipping and negative deltaPSI to promote inclusion
        if flip_needed == 'yes':
            df['IncLevelDifference'] = df.IncLevelDifference * -1    
        return(df)

    filt_df = test_filter(test_df,reads,fdr,dpsi,flip_needed)
    
    print(len(filt_df),'number_of_SE_events_post_reads_fdr_dpsi')
    
    gene_df = pd.read_table(cuffdiff_file)
    gene_df['GeneID'] = gene_df.gene_id.str.split('.').str[0]

    #Merge splice and gene
    splice_gene_merge = filt_df.merge(gene_df,on='GeneID')
    
    #filt by FPKM
    splice_gene_filt = splice_gene_merge[(splice_gene_merge.value_1 > fpkm)|(splice_gene_merge.value_2 > fpkm)]
    
    print(len(splice_gene_filt),'number_of_SE_events_post_FPKM')
    
    gencode_v24 = pbt.BedTool('../annotations/gencode_extracted_regions/prepare_collapsed_exons/gencode.v24lift37.basic_exons.final_collapse.bed')

    def filter_out_events_with_intervening_exons(df,exon_bed):
        #Get the intron bed
        bed_df = df[['chr','upstreamEE','downstreamES','splice_id','IncLevelDifference','strand']]
        bed_bt = pbt.BedTool.from_dataframe(bed_df)
        #get the exon bed
        exon_bed = pbt.BedTool(exon_bed)
        #Get the number of whole overlap intersections per event
        intersect = bed_bt.intersect(exon_bed,c=True,s=True,F=1)
        intersect_names = bed_bt.intersect(exon_bed,wao=True,s=True,F=1)
        #make into df
        cols = ['chr','upstreamEE','downstreamES','splice_id','IncLevelDifference','strand','num_intersect']
        intersect_df = intersect.to_dataframe(names=cols)
        cols = ['chr','upstreamEE','downstreamES','splice_id','IncLevelDifference','strand','a','b','c','d','e','f','g']
        intersect_names_df = intersect_names.to_dataframe(names=cols)
        #collect the ones with only one overlap
        one_overlap = intersect_df[intersect_df.num_intersect == 1]
        return(intersect_df,intersect_names_df,one_overlap)

    if one_variable_only == 'yes':
        intersect_df,intersect_names_df,one_overlap = filter_out_events_with_intervening_exons(splice_gene_filt,gencode_v24)
        print(len(one_overlap),'number_of_SE_events_with_one_V_exon')
    else:
        one_overlap = splice_gene_filt
    
    def run_rmats2sashimiplot(select_df,orig_df,df_name,bam1_list,bam2_list,label1,label2,exon_scale,intron_scale,sample_frac,sample_seed):
        cols = ['ID','GeneID','geneSymbol','chr','strand','exonStart_0base','exonEnd','upstreamES','upstreamEE','downstreamES','downstreamEE','ID','IJC_SAMPLE_1','SJC_SAMPLE_1','IJC_SAMPLE_2','SJC_SAMPLE_2','IncFormLen','SkipFormLen','PValue','FDR','IncLevel1','IncLevel2','IncLevelDifference']
        full_df = orig_df[orig_df.splice_id.isin(select_df.splice_id)]
        os.system('mkdir RMATS_VALIDATION/{}'.format(df_name))
        
        #save the full df
        full_df.to_csv('RMATS_VALIDATION/'+df_name+'/'+df_name+'.rmats_filt.full.txt',sep='\t',index=False)
        
        #Take a sample percent of these events to make rMATS2SashimiPlot
        df = full_df.sample(frac=sample_frac,random_state=sample_seed)

        df.to_csv('RMATS_VALIDATION/'+df_name+'/'+df_name+'.rmats_filt.sample.txt',sep='\t',index=False)
        fn = 'RMATS_VALIDATION/'+df_name+'/'+df_name+'.rmats_filt.sample.txt'
        if run_sashimi == 'yes':
            cmd = 'rmats2sashimiplot --b1 {} --b2 {} -t SE -e {} --l1 {} --l2 {} --exon_s {} --intron_s {} -o {}'.format(bam1_list,bam2_list,fn,label1,label2,exon_scale,intron_scale,'RMATS_VALIDATION/'+df_name)
            os.system(cmd)    
        
    run_rmats2sashimiplot(one_overlap,filt_df,df_name,bam1_list,bam2_list,label1,label2,exon_scale,intron_scale,sample_frac,sample_seed)

In [46]:
ucsc_canon = pbt.BedTool('../annotations/ucsc_known_canonical.bed6')
gencode_v24 = pbt.BedTool('../annotations/gencode_extracted_regions/prepare_collapsed_exons/gencode.v24lift37.basic_exons.final_collapse.bed')

# Running HTE

In [ ]:
#Not complex

In [104]:
#Remember - these are all gencodev24 basic with rMATS c=0.0001
rmats_parsing_file = 'rMATS_parsing_turbo/hte_turbo_loose_all_SE_FDR_1.0_dPSI_0.0_read_cutoff_0.0.txt'
cuffdiff_file = '../cuffdiff_runs/HTE_cuffdiff/gene_exp.diff'

fdr = 0.05
dpsi = 0.15
reads = 10
fpkm = 5
flip_needed = 'yes'
one_variable_only = 'yes'
run_sashimi = 'yes'

exon_scale = 1
intron_scale = 4
sample_frac = 0.1
sample_seed = 10

bam1_list = '../star_runs/HTE1Aligned.sortedByCoord.out.bam,../star_runs/HTE3Aligned.sortedByCoord.out.bam'
bam2_list = '../star_runs/HTE2Aligned.sortedByCoord.out.bam,../star_runs/HTE4Aligned.sortedByCoord.out.bam'
df_name = 'HTE_run1'
label1 = 'HTE_WT'
label2 = 'HTE_KD'

master_rmats_validation(rmats_parsing_file,cuffdiff_file,df_name,bam1_list,bam2_list,label1,label2,exon_scale,intron_scale,sample_frac,sample_seed,flip_needed,fdr,dpsi,reads,one_variable_only,run_sashimi)

(58079, 'number_of_SE_events')
(528, 'number_of_SE_events_post_reads_fdr_dpsi')
(349, 'number_of_SE_events_post_FPKM')
(168, 'number_of_SE_events_with_one_V_exon')


In [ ]:
#Complex

In [105]:
rmats_parsing_file = 'rMATS_parsing_turbo/hte_turbo_loose_all_SE_FDR_1.0_dPSI_0.0_read_cutoff_0.0.txt'
cuffdiff_file = '../cuffdiff_runs/HTE_cuffdiff/gene_exp.diff'

fdr = 0.05
dpsi = 0.15
reads = 10
fpkm = 5
flip_needed = 'yes'
one_variable_only = 'no'
run_sashimi = 'yes'


exon_scale = 1
intron_scale = 4
sample_frac = 0.1
sample_seed = 10

bam1_list = '../star_runs/HTE1Aligned.sortedByCoord.out.bam,../star_runs/HTE3Aligned.sortedByCoord.out.bam'
bam2_list = '../star_runs/HTE2Aligned.sortedByCoord.out.bam,../star_runs/HTE4Aligned.sortedByCoord.out.bam'
df_name = 'HTE_run1_complex'
label1 = 'HTE_WT'
label2 = 'HTE_KD'

master_rmats_validation(rmats_parsing_file,cuffdiff_file,df_name,bam1_list,bam2_list,label1,label2,exon_scale,intron_scale,sample_frac,sample_seed,flip_needed,fdr,dpsi,reads,one_variable_only,run_sashimi)

(58079, 'number_of_SE_events')
(528, 'number_of_SE_events_post_reads_fdr_dpsi')
(349, 'number_of_SE_events_post_FPKM')


In [134]:
#original
rmats_parsing_file = 'rMATS_parsing_turbo/hte_turbo_original_all_SE_FDR_1.0_dPSI_0.0_read_cutoff_0.0.txt'
cuffdiff_file = '../cuffdiff_runs/HTE_cuffdiff/gene_exp.diff'

fdr = 0.05
dpsi = 0.15
reads = 10
fpkm = 5
flip_needed = 'yes'
one_variable_only = 'no'
run_sashimi = 'yes'


exon_scale = 1
intron_scale = 4
sample_frac = 0.1
sample_seed = 10

bam1_list = '../star_runs/HTE1Aligned.sortedByCoord.out.bam,../star_runs/HTE3Aligned.sortedByCoord.out.bam'
bam2_list = '../star_runs/HTE2Aligned.sortedByCoord.out.bam,../star_runs/HTE4Aligned.sortedByCoord.out.bam'
df_name = 'HTE_run1_original'
label1 = 'HTE_WT'
label2 = 'HTE_KD'

master_rmats_validation(rmats_parsing_file,cuffdiff_file,df_name,bam1_list,bam2_list,label1,label2,exon_scale,intron_scale,sample_frac,sample_seed,flip_needed,fdr,dpsi,reads,one_variable_only,run_sashimi)

(76188, 'number_of_SE_events')
(780, 'number_of_SE_events_post_reads_fdr_dpsi')
(528, 'number_of_SE_events_post_FPKM')


In [145]:
#with TSL1 and c = 0.0001
rmats_parsing_file = 'rMATS_parsing_turbo/hte_turbo_tsl1_all_SE_FDR_1.0_dPSI_0.0_read_cutoff_0.0.txt'
cuffdiff_file = '../cuffdiff_runs/HTE_cuffdiff/gene_exp.diff'

fdr = 0.05
dpsi = 0.15
reads = 10
fpkm = 5
flip_needed = 'yes'
one_variable_only = 'no'
run_sashimi = 'yes'

exon_scale = 1
intron_scale = 4
sample_frac = 0.1
sample_seed = 10

bam1_list = '../star_runs/HTE1Aligned.sortedByCoord.out.bam,../star_runs/HTE3Aligned.sortedByCoord.out.bam'
bam2_list = '../star_runs/HTE2Aligned.sortedByCoord.out.bam,../star_runs/HTE4Aligned.sortedByCoord.out.bam'
df_name = 'HTE_run1_tsl1'
label1 = 'HTE_WT'
label2 = 'HTE_KD'

master_rmats_validation(rmats_parsing_file,cuffdiff_file,df_name,bam1_list,bam2_list,label1,label2,exon_scale,intron_scale,sample_frac,sample_seed,flip_needed,fdr,dpsi,reads,one_variable_only,run_sashimi)

(47450, 'number_of_SE_events')
(347, 'number_of_SE_events_post_reads_fdr_dpsi')
(241, 'number_of_SE_events_post_FPKM')


In [143]:
#With gencode basic, but c = 0.05
rmats_parsing_file = 'rMATS_parsing_turbo/hte_turbo_strict_all_SE_FDR_1.0_dPSI_0.0_read_cutoff_0.0.txt'
cuffdiff_file = '../cuffdiff_runs/HTE_cuffdiff/gene_exp.diff'

fdr = 0.05
dpsi = 0.15
reads = 10
fpkm = 5
flip_needed = 'yes'
one_variable_only = 'no'
run_sashimi = 'yes'

exon_scale = 1
intron_scale = 4
sample_frac = 0.1
sample_seed = 10

bam1_list = '../star_runs/HTE1Aligned.sortedByCoord.out.bam,../star_runs/HTE3Aligned.sortedByCoord.out.bam'
bam2_list = '../star_runs/HTE2Aligned.sortedByCoord.out.bam,../star_runs/HTE4Aligned.sortedByCoord.out.bam'
df_name = 'HTE_strict'
label1 = 'HTE_WT'
label2 = 'HTE_KD'

master_rmats_validation(rmats_parsing_file,cuffdiff_file,df_name,bam1_list,bam2_list,label1,label2,exon_scale,intron_scale,sample_frac,sample_seed,flip_needed,fdr,dpsi,reads,one_variable_only,run_sashimi)

(58079, 'number_of_SE_events')
(263, 'number_of_SE_events_post_reads_fdr_dpsi')
(210, 'number_of_SE_events_post_FPKM')


In [127]:
intersection = pd.read_table('../../../systematic_analysis_of_sam_data/hte_rmats_whippet_intersection.txt')
intersection.head()

,ID,GeneID,geneSymbol,chr,strand,exonStart_0base,exonEnd,upstreamES,upstreamEE,downstreamES,downstreamEE,ID.1,IJC_SAMPLE_1,SJC_SAMPLE_1,IJC_SAMPLE_2,SJC_SAMPLE_2,IncFormLen,SkipFormLen,PValue,FDR,IncLevel1,IncLevel2,IncLevelDifference,splice_id,splicing_direction,IC_SAMPLE_1_mean,SC_SAMPLE_1_mean,SAMPLE_1_mean_total,IC_SAMPLE_2_mean,SC_SAMPLE_2_mean,SAMPLE_2_mean_total,IC_SAMPLE_1_REP1,SC_SAMPLE_1_REP1,IC_SAMPLE_2_REP1,SC_SAMPLE_2_REP1,IC_SAMPLE_1_REP2,SC_SAMPLE_1_REP2,IC_SAMPLE_2_REP2,SC_SAMPLE_2_REP2,IncLevel1_mean,IncLevel2_mean,INC_LEVEL_SAMPLE_1_REP1,INC_LEVEL_SAMPLE_2_REP1,INC_LEVEL_SAMPLE_1_REP2,INC_LEVEL_SAMPLE_2_REP2,v_id,combo_junctions,start,start_1_base,new_id
0,57993,ENSG00000092841,MYL6,chr12,+,56554409,56554454,56554026,56554104,56555170,56555334,57993,"11445,11646","5721,5224","9599,7361","8815,8173",143,99,7.827072e-14,2.404552e-11,"0.581,0.607","0.43,0.384",-0.187,SE:chr12:56554409-56554454:56554026-56554104:56555170-56555334:+:MYL6,skip,11545.5,5472.5,17018.0,8480.0,8494.0,16974.0,11445,5721,9599,8815,11646,5224,7361,8173,0.5940,0.4070,0.581,0.430,0.607,0.384,chr12:56554409-56554454:+:MYL6:ENSG00000092841,33992.0,56554409,56554410,chr12:56554410-56554454
1,34798,ENSG00000060138,YBX3,chr12,-,10862506,10862713,10856621,10856747,10865809,10865932,34798,"3876,3562","2874,2614","6313,6427","2778,2238",198,99,7.801011e-10,1.163099e-07,"0.403,0.405","0.532,0.589",0.156,SE:chr12:10862506-10862713:10856621-10856747:10865809-10865932:-:YBX3,include,3719.0,2744.0,6463.0,6370.0,2508.0,8878.0,3876,2874,6313,2778,3562,2614,6427,2238,0.4040,0.5605,0.403,0.532,0.405,0.589,chr12:10862506-10862713:-:YBX3:ENSG00000060138,15341.0,10862506,10862507,chr12:10862507-10862713
2,27885,ENSG00000084234,APLP2,chr11,+,129993506,129993674,129992199,129992408,129996594,129996725,27885,"5438,5059","1546,1799","5503,5007","440,385",198,99,0.000000e+00,0.000000e+00,"0.638,0.584","0.862,0.867",0.254,SE:chr11:129993506-129993674:129992199-129992408:129996594-129996725:+:APLP2,include,5248.5,1672.5,6921.0,5255.0,412.5,5667.5,5438,1546,5503,440,5059,1799,5007,385,0.6110,0.8645,0.638,0.862,0.584,0.867,chr11:129993506-129993674:+:APLP2:ENSG00000084234,12588.5,129993506,129993507,chr11:129993507-129993674
3,23471,ENSG00000155366,RHOC,chr1,-,113247721,113247790,113246114,113246428,113249699,113249736,23471,"2698,2603","1925,1538","5818,5079","1508,992",167,99,0.000000e+00,0.000000e+00,"0.454,0.501","0.696,0.752",0.246,SE:chr1:113247721-113247790:113246114-113246428:113249699-113249736:-:RHOC,include,2650.5,1731.5,4382.0,5448.5,1250.0,6698.5,2698,1925,5818,1508,2603,1538,5079,992,0.4775,0.7240,0.454,0.696,0.501,0.752,chr1:113247721-113247790:-:RHOC:ENSG00000155366,11080.5,113247721,113247722,chr1:113247722-113247790
4,13505,ENSG00000136238,RAC1,chr7,+,6438292,6438349,6431554,6431672,6439756,6439819,13505,"1014,1164","4011,3656","2485,2760","2543,2262",155,99,0.000000e+00,0.000000e+00,"0.139,0.169","0.384,0.438",0.257,SE:chr7:6438292-6438349:6431554-6431672:6439756-6439819:+:RAC1,include,1089.0,3833.5,4922.5,2622.5,2402.5,5025.0,1014,4011,2485,2543,1164,3656,2760,2262,0.1540,0.4110,0.139,0.384,0.169,0.438,chr7:6438292-6438349:+:RAC1:ENSG00000136238,9947.5,6438292,6438293,chr7:6438293-6438349


In [146]:
hte_orig_parsing = pd.read_table('rMATS_parsing_turbo/hte_turbo_original_all_SE_FDR_1.0_dPSI_0.0_read_cutoff_0.0.txt')
hte_new_parsing = pd.read_table('rMATS_parsing_turbo/')

In [132]:
hte_orig_parsing_intersection = hte_orig_parsing[hte_orig_parsing.splice_id.isin(intersection.splice_id)]
hte_orig_parsing_intersection.to_csv('rMATS_parsing_turbo/hte_orig_whippet_all_SE.txt',sep='\t',index=False)

In [139]:
check = pd.read_table('rMATS_parsing_turbo/hte_orig_whippet_all_SE.txt')
check.head()

,ID,GeneID,geneSymbol,chr,strand,exonStart_0base,exonEnd,upstreamES,upstreamEE,downstreamES,downstreamEE,ID.1,IJC_SAMPLE_1,SJC_SAMPLE_1,IJC_SAMPLE_2,SJC_SAMPLE_2,IncFormLen,SkipFormLen,PValue,FDR,IncLevel1,IncLevel2,IncLevelDifference,splice_id,splicing_direction,IC_SAMPLE_1_mean,SC_SAMPLE_1_mean,SAMPLE_1_mean_total,IC_SAMPLE_2_mean,SC_SAMPLE_2_mean,SAMPLE_2_mean_total,IC_SAMPLE_1_REP1,SC_SAMPLE_1_REP1,IC_SAMPLE_2_REP1,SC_SAMPLE_2_REP1,IC_SAMPLE_1_REP2,SC_SAMPLE_1_REP2,IC_SAMPLE_2_REP2,SC_SAMPLE_2_REP2,IncLevel1_mean,IncLevel2_mean,INC_LEVEL_SAMPLE_1_REP1,INC_LEVEL_SAMPLE_2_REP1,INC_LEVEL_SAMPLE_1_REP2,INC_LEVEL_SAMPLE_2_REP2
0,652,ENSG00000101363,MANBAL,chr20,+,35927165,35927282,35918065,35918089,35929610,35929816,652,"507,478","161,110","277,216","25,14",198,99,1.735495e-10,2.991491e-08,"0.612,0.685","0.847,0.885",-0.217,SE:chr20:35927165-35927282:35918065-35918089:35929610-35929816:+:MANBAL,include,492.5,135.5,628.0,246.5,19.5,266.0,507,161,277,25,478,110,216,14,0.6485,0.8660,0.612,0.847,0.685,0.885
1,3222,ENSG00000049759,NEDD4L,chr18,+,56002709,56002769,56001049,56001124,56008269,56008401,3222,"182,217","502,453","220,239","241,222",158,99,9.099199e-11,1.646674e-08,"0.185,0.231","0.364,0.403",-0.175,SE:chr18:56002709-56002769:56001049-56001124:56008269-56008401:+:NEDD4L,include,199.5,477.5,677.0,229.5,231.5,461.0,182,502,220,241,217,453,239,222,0.2080,0.3835,0.185,0.364,0.231,0.403
2,3948,ENSG00000130714,POMT1,chr9,+,134379575,134379727,134378311,134378460,134381500,134381607,3948,"93,96","9,28","158,144","2,3",198,99,5.988043e-08,5.924897e-06,"0.838,0.632","0.975,0.96",-0.233,SE:chr9:134379575-134379727:134378311-134378460:134381500-134381607:+:POMT1,include,94.5,18.5,113.0,151.0,2.5,153.5,93,9,158,2,96,28,144,3,0.7350,0.9675,0.838,0.975,0.632,0.960
3,4831,ENSG00000174151,CYB561D1,chr1,+,110037141,110037245,110036720,110036909,110037764,110037802,4831,"42,32","8,7","51,28","25,21",198,99,1.737617e-03,3.276060e-02,"0.724,0.696","0.505,0.4",0.257,SE:chr1:110037141-110037245:110036720-110036909:110037764-110037802:+:CYB561D1,skip,37.0,7.5,44.5,39.5,23.0,62.5,42,8,51,25,32,7,28,21,0.7100,0.4525,0.724,0.505,0.696,0.400
4,6143,ENSG00000153827,TRIP12,chr2,-,230725121,230725247,230723487,230724290,230744697,230744844,6143,"629,655","282,305","1152,924","109,117",198,99,0.000000e+00,0.000000e+00,"0.527,0.518","0.841,0.798",-0.297,SE:chr2:230725121-230725247:230723487-230724290:230744697-230744844:-:TRIP12,include,642.0,293.5,935.5,1038.0,113.0,1151.0,629,282,1152,109,655,305,924,117,0.5225,0.8195,0.527,0.841,0.518,0.798


In [142]:
#Intersection of rmats original and whippet
#Wait - this is also the intersection of new too... doesnt' make sense
rmats_parsing_file = 'rMATS_parsing_turbo/hte_orig_whippet_all_SE.txt'
cuffdiff_file = '../cuffdiff_runs/HTE_cuffdiff/gene_exp.diff'

fdr = 0.05
dpsi = 0.15
reads = 10
fpkm = 5
flip_needed = 'yes'
one_variable_only = 'no'
run_sashimi = 'yes'


exon_scale = 1
intron_scale = 4
sample_frac = 0.1
sample_seed = 10

bam1_list = '../star_runs/HTE1Aligned.sortedByCoord.out.bam,../star_runs/HTE3Aligned.sortedByCoord.out.bam'
bam2_list = '../star_runs/HTE2Aligned.sortedByCoord.out.bam,../star_runs/HTE4Aligned.sortedByCoord.out.bam'
df_name = 'HTE_whippet_intersection'
label1 = 'HTE_WT'
label2 = 'HTE_KD'

master_rmats_validation(rmats_parsing_file,cuffdiff_file,df_name,bam1_list,bam2_list,label1,label2,exon_scale,intron_scale,sample_frac,sample_seed,flip_needed,fdr,dpsi,reads,one_variable_only,run_sashimi)

(172, 'number_of_SE_events')
(172, 'number_of_SE_events_post_reads_fdr_dpsi')
(172, 'number_of_SE_events_post_FPKM')


# Running HTM

In [50]:
rmats_parsing_file = 'rMATS_parsing_turbo/htm_turbo_loose_all_SE_FDR_1.0_dPSI_0.0_read_cutoff_0.0.txt'
cuffdiff_file = '../cuffdiff_runs/HTM_cuffdiff/gene_exp.diff'

fdr = 0.05
dpsi = 0.15
reads = 10
fpkm = 5
flip_needed = 'yes'
one_variable_only = 'yes'
run_sashimi = 'yes'


exon_scale = 1
intron_scale = 4
sample_frac = 0.1
sample_seed = 10

bam1_list = '../star_runs/HTM1Aligned.sortedByCoord.out.bam,../star_runs/HTM3Aligned.sortedByCoord.out.bam'
bam2_list = '../star_runs/HTM2Aligned.sortedByCoord.out.bam,../star_runs/HTM4Aligned.sortedByCoord.out.bam'
df_name = 'HTM_run1'
label1 = 'HTM_WT'
label2 = 'HTM_KD'

master_rmats_validation(rmats_parsing_file,cuffdiff_file,df_name,bam1_list,bam2_list,label1,label2,exon_scale,intron_scale,sample_frac,sample_seed,flip_needed,fdr,dpsi,reads,one_variable_only,run_sashimi)

(71051, 'number_of_SE_events')
(412, 'number_of_SE_events_post_reads_fdr_dpsi')
(288, 'number_of_SE_events_post_FPKM')
(140, 'number_of_SE_events_with_one_V_exon')


In [54]:
rmats_parsing_file = 'rMATS_parsing_turbo/htm_turbo_loose_all_SE_FDR_1.0_dPSI_0.0_read_cutoff_0.0.txt'
cuffdiff_file = '../cuffdiff_runs/HTM_cuffdiff/gene_exp.diff'

fdr = 0.05
dpsi = 0.15
reads = 10
fpkm = 5
flip_needed = 'yes'
one_variable_only = 'no'
run_sashimi = 'yes'


exon_scale = 1
intron_scale = 4
sample_frac = 0.1
sample_seed = 10

bam1_list = '../star_runs/HTM1Aligned.sortedByCoord.out.bam,../star_runs/HTM3Aligned.sortedByCoord.out.bam'
bam2_list = '../star_runs/HTM2Aligned.sortedByCoord.out.bam,../star_runs/HTM4Aligned.sortedByCoord.out.bam'
df_name = 'HTM_run1_complex'
label1 = 'HTM_WT'
label2 = 'HTM_KD'

master_rmats_validation(rmats_parsing_file,cuffdiff_file,df_name,bam1_list,bam2_list,label1,label2,exon_scale,intron_scale,sample_frac,sample_seed,flip_needed,fdr,dpsi,reads,one_variable_only,run_sashimi)

(71051, 'number_of_SE_events')
(412, 'number_of_SE_events_post_reads_fdr_dpsi')
(288, 'number_of_SE_events_post_FPKM')


# Running HTEMT

In [52]:
rmats_parsing_file = 'rMATS_parsing_turbo/htemt_turbo_loose_all_SE_FDR_1.0_dPSI_0.0_read_cutoff_0.0.txt'
cuffdiff_file = '../cuffdiff_runs/HTEMT_cuffdiff/gene_exp.diff'

fdr = 0.05
dpsi = 0.15
reads = 10
fpkm = 5
flip_needed = 'no'
one_variable_only = 'yes'
run_sashimi = 'yes'


exon_scale = 1
intron_scale = 4
sample_frac = 0.1
sample_seed = 10

bam1_list = '../star_runs/HTE1Aligned.sortedByCoord.out.bam,../star_runs/HTE3Aligned.sortedByCoord.out.bam'
bam2_list = '../star_runs/HTM1Aligned.sortedByCoord.out.bam,../star_runs/HTM3Aligned.sortedByCoord.out.bam'
df_name = 'HTEMT_run1'
label1 = 'HTEMT_EPI'
label2 = 'HTEMT_MES'

master_rmats_validation(rmats_parsing_file,cuffdiff_file,df_name,bam1_list,bam2_list,label1,label2,exon_scale,intron_scale,sample_frac,sample_seed,flip_needed,fdr,dpsi,reads,one_variable_only,run_sashimi)

(67195, 'number_of_SE_events')
(811, 'number_of_SE_events_post_reads_fdr_dpsi')
(594, 'number_of_SE_events_post_FPKM')
(336, 'number_of_SE_events_with_one_V_exon')


In [55]:
rmats_parsing_file = 'rMATS_parsing_turbo/htemt_turbo_loose_all_SE_FDR_1.0_dPSI_0.0_read_cutoff_0.0.txt'
cuffdiff_file = '../cuffdiff_runs/HTEMT_cuffdiff/gene_exp.diff'

fdr = 0.05
dpsi = 0.15
reads = 10
fpkm = 5
flip_needed = 'yes'
one_variable_only = 'no'
run_sashimi = 'yes'


exon_scale = 1
intron_scale = 4
sample_frac = 0.1
sample_seed = 10

bam1_list = '../star_runs/HTE1Aligned.sortedByCoord.out.bam,../star_runs/HTE3Aligned.sortedByCoord.out.bam'
bam2_list = '../star_runs/HTM1Aligned.sortedByCoord.out.bam,../star_runs/HTM3Aligned.sortedByCoord.out.bam'
df_name = 'HTEMT_run1_complex'
label1 = 'HTEMT_EPI'
label2 = 'HTEMT_MES'

master_rmats_validation(rmats_parsing_file,cuffdiff_file,df_name,bam1_list,bam2_list,label1,label2,exon_scale,intron_scale,sample_frac,sample_seed,flip_needed,fdr,dpsi,reads,one_variable_only,run_sashimi)

(67195, 'number_of_SE_events')
(811, 'number_of_SE_events_post_reads_fdr_dpsi')
(594, 'number_of_SE_events_post_FPKM')


# Testing different cutoffs

In [ ]:
#dPSI

In [109]:
#What should I test for one sample set? Let's see how much we get from an example set HTE
rmats_parsing_file = 'rMATS_parsing_turbo/hte_turbo_tsl1_c05_all_SE_FDR_1.0_dPSI_0.0_read_cutoff_0.0.txt'
cuffdiff_file = '../cuffdiff_runs/HTE_cuffdiff/gene_exp.diff'

fdr = 0.05
dpsi = 0.1
reads = 10
fpkm = 5
cutoff_string = '_'.join([str(fdr),str(dpsi),str(reads),str(fpkm)])

flip_needed = 'yes'
one_variable_only = 'no'
run_sashimi = 'no'

exon_scale = 1
intron_scale = 4
sample_frac = 0.1
sample_seed = 10

bam1_list = '../star_runs/HTE1Aligned.sortedByCoord.out.bam,../star_runs/HTE3Aligned.sortedByCoord.out.bam'
bam2_list = '../star_runs/HTE2Aligned.sortedByCoord.out.bam,../star_runs/HTE4Aligned.sortedByCoord.out.bam'
df_name = 'HTE_loose_'+cutoff_string
label1 = 'HTE_WT'
label2 = 'HTE_KD'

master_rmats_validation(rmats_parsing_file,cuffdiff_file,df_name,bam1_list,bam2_list,label1,label2,exon_scale,intron_scale,sample_frac,sample_seed,flip_needed,fdr,dpsi,reads,one_variable_only,run_sashimi)
print('\n')
dpsi = 0.15
master_rmats_validation(rmats_parsing_file,cuffdiff_file,df_name,bam1_list,bam2_list,label1,label2,exon_scale,intron_scale,sample_frac,sample_seed,flip_needed,fdr,dpsi,reads,one_variable_only,run_sashimi)
print('\n')
dpsi = 0.2
master_rmats_validation(rmats_parsing_file,cuffdiff_file,df_name,bam1_list,bam2_list,label1,label2,exon_scale,intron_scale,sample_frac,sample_seed,flip_needed,fdr,dpsi,reads,one_variable_only,run_sashimi)

(47450, 'number_of_SE_events')
(198, 'number_of_SE_events_post_reads_fdr_dpsi')
(173, 'number_of_SE_events_post_FPKM')


(47450, 'number_of_SE_events')
(163, 'number_of_SE_events_post_reads_fdr_dpsi')
(139, 'number_of_SE_events_post_FPKM')


(47450, 'number_of_SE_events')
(107, 'number_of_SE_events_post_reads_fdr_dpsi')
(88, 'number_of_SE_events_post_FPKM')


In [110]:
#FDR

In [111]:
#What should I test for one sample set? Let's see how much we get from an example set HTE
rmats_parsing_file = 'rMATS_parsing_turbo/hte_turbo_tsl1_c05_all_SE_FDR_1.0_dPSI_0.0_read_cutoff_0.0.txt'
cuffdiff_file = '../cuffdiff_runs/HTE_cuffdiff/gene_exp.diff'

fdr = 0.1
dpsi = 0.1
reads = 10
fpkm = 5
cutoff_string = '_'.join([str(fdr),str(dpsi),str(reads),str(fpkm)])

flip_needed = 'yes'
one_variable_only = 'no'
run_sashimi = 'no'

exon_scale = 1
intron_scale = 4
sample_frac = 0.1
sample_seed = 10

bam1_list = '../star_runs/HTE1Aligned.sortedByCoord.out.bam,../star_runs/HTE3Aligned.sortedByCoord.out.bam'
bam2_list = '../star_runs/HTE2Aligned.sortedByCoord.out.bam,../star_runs/HTE4Aligned.sortedByCoord.out.bam'
df_name = 'HTE_loose_'+cutoff_string
label1 = 'HTE_WT'
label2 = 'HTE_KD'

master_rmats_validation(rmats_parsing_file,cuffdiff_file,df_name,bam1_list,bam2_list,label1,label2,exon_scale,intron_scale,sample_frac,sample_seed,flip_needed,fdr,dpsi,reads,one_variable_only,run_sashimi)
print('\n')
fdr = 0.05
master_rmats_validation(rmats_parsing_file,cuffdiff_file,df_name,bam1_list,bam2_list,label1,label2,exon_scale,intron_scale,sample_frac,sample_seed,flip_needed,fdr,dpsi,reads,one_variable_only,run_sashimi)
print('\n')
fdr = 0.01
master_rmats_validation(rmats_parsing_file,cuffdiff_file,df_name,bam1_list,bam2_list,label1,label2,exon_scale,intron_scale,sample_frac,sample_seed,flip_needed,fdr,dpsi,reads,one_variable_only,run_sashimi)

(47450, 'number_of_SE_events')
(224, 'number_of_SE_events_post_reads_fdr_dpsi')
(190, 'number_of_SE_events_post_FPKM')


(47450, 'number_of_SE_events')
(198, 'number_of_SE_events_post_reads_fdr_dpsi')
(173, 'number_of_SE_events_post_FPKM')


(47450, 'number_of_SE_events')
(156, 'number_of_SE_events_post_reads_fdr_dpsi')
(140, 'number_of_SE_events_post_FPKM')


In [112]:
#JUNCTION

In [113]:
#What should I test for one sample set? Let's see how much we get from an example set HTE
rmats_parsing_file = 'rMATS_parsing_turbo/hte_turbo_tsl1_c05_all_SE_FDR_1.0_dPSI_0.0_read_cutoff_0.0.txt'
cuffdiff_file = '../cuffdiff_runs/HTE_cuffdiff/gene_exp.diff'

fdr = 0.1
dpsi = 0.1
reads = 10
fpkm = 5
cutoff_string = '_'.join([str(fdr),str(dpsi),str(reads),str(fpkm)])

flip_needed = 'yes'
one_variable_only = 'no'
run_sashimi = 'no'

exon_scale = 1
intron_scale = 4
sample_frac = 0.1
sample_seed = 10

bam1_list = '../star_runs/HTE1Aligned.sortedByCoord.out.bam,../star_runs/HTE3Aligned.sortedByCoord.out.bam'
bam2_list = '../star_runs/HTE2Aligned.sortedByCoord.out.bam,../star_runs/HTE4Aligned.sortedByCoord.out.bam'
df_name = 'HTE_loose_'+cutoff_string
label1 = 'HTE_WT'
label2 = 'HTE_KD'

master_rmats_validation(rmats_parsing_file,cuffdiff_file,df_name,bam1_list,bam2_list,label1,label2,exon_scale,intron_scale,sample_frac,sample_seed,flip_needed,fdr,dpsi,reads,one_variable_only,run_sashimi)
print('\n')
reads = 20
master_rmats_validation(rmats_parsing_file,cuffdiff_file,df_name,bam1_list,bam2_list,label1,label2,exon_scale,intron_scale,sample_frac,sample_seed,flip_needed,fdr,dpsi,reads,one_variable_only,run_sashimi)
print('\n')
reads = 40
master_rmats_validation(rmats_parsing_file,cuffdiff_file,df_name,bam1_list,bam2_list,label1,label2,exon_scale,intron_scale,sample_frac,sample_seed,flip_needed,fdr,dpsi,reads,one_variable_only,run_sashimi)

(47450, 'number_of_SE_events')
(224, 'number_of_SE_events_post_reads_fdr_dpsi')
(190, 'number_of_SE_events_post_FPKM')


(47450, 'number_of_SE_events')
(217, 'number_of_SE_events_post_reads_fdr_dpsi')
(189, 'number_of_SE_events_post_FPKM')


(47450, 'number_of_SE_events')
(200, 'number_of_SE_events_post_reads_fdr_dpsi')
(184, 'number_of_SE_events_post_FPKM')


In [114]:
#FPKM

In [115]:
#What should I test for one sample set? Let's see how much we get from an example set HTE
rmats_parsing_file = 'rMATS_parsing_turbo/hte_turbo_tsl1_c05_all_SE_FDR_1.0_dPSI_0.0_read_cutoff_0.0.txt'
cuffdiff_file = '../cuffdiff_runs/HTE_cuffdiff/gene_exp.diff'

fdr = 0.05
dpsi = 0.1
reads = 10
fpkm = 3
cutoff_string = '_'.join([str(fdr),str(dpsi),str(reads),str(fpkm)])

flip_needed = 'yes'
one_variable_only = 'no'
run_sashimi = 'no'

exon_scale = 1
intron_scale = 4
sample_frac = 0.1
sample_seed = 10

bam1_list = '../star_runs/HTE1Aligned.sortedByCoord.out.bam,../star_runs/HTE3Aligned.sortedByCoord.out.bam'
bam2_list = '../star_runs/HTE2Aligned.sortedByCoord.out.bam,../star_runs/HTE4Aligned.sortedByCoord.out.bam'
df_name = 'HTE_loose_'+cutoff_string
label1 = 'HTE_WT'
label2 = 'HTE_KD'

master_rmats_validation(rmats_parsing_file,cuffdiff_file,df_name,bam1_list,bam2_list,label1,label2,exon_scale,intron_scale,sample_frac,sample_seed,flip_needed,fdr,dpsi,reads,one_variable_only,run_sashimi)
print('\n')
fpkm = 5
master_rmats_validation(rmats_parsing_file,cuffdiff_file,df_name,bam1_list,bam2_list,label1,label2,exon_scale,intron_scale,sample_frac,sample_seed,flip_needed,fdr,dpsi,reads,one_variable_only,run_sashimi)
print('\n')
fpkm = 10
master_rmats_validation(rmats_parsing_file,cuffdiff_file,df_name,bam1_list,bam2_list,label1,label2,exon_scale,intron_scale,sample_frac,sample_seed,flip_needed,fdr,dpsi,reads,one_variable_only,run_sashimi)

(47450, 'number_of_SE_events')
(198, 'number_of_SE_events_post_reads_fdr_dpsi')
(188, 'number_of_SE_events_post_FPKM')


(47450, 'number_of_SE_events')
(198, 'number_of_SE_events_post_reads_fdr_dpsi')
(173, 'number_of_SE_events_post_FPKM')


(47450, 'number_of_SE_events')
(198, 'number_of_SE_events_post_reads_fdr_dpsi')
(145, 'number_of_SE_events_post_FPKM')


In [116]:
#One Variable
#What should I test for one sample set? Let's see how much we get from an example set HTE
rmats_parsing_file = 'rMATS_parsing_turbo/hte_turbo_tsl1_c05_all_SE_FDR_1.0_dPSI_0.0_read_cutoff_0.0.txt'
cuffdiff_file = '../cuffdiff_runs/HTE_cuffdiff/gene_exp.diff'

fdr = 0.05
dpsi = 0.1
reads = 10
fpkm = 5
cutoff_string = '_'.join([str(fdr),str(dpsi),str(reads),str(fpkm)])

flip_needed = 'yes'
one_variable_only = 'no'
run_sashimi = 'no'

exon_scale = 1
intron_scale = 4
sample_frac = 0.1
sample_seed = 10

bam1_list = '../star_runs/HTE1Aligned.sortedByCoord.out.bam,../star_runs/HTE3Aligned.sortedByCoord.out.bam'
bam2_list = '../star_runs/HTE2Aligned.sortedByCoord.out.bam,../star_runs/HTE4Aligned.sortedByCoord.out.bam'
df_name = 'HTE_loose_'+cutoff_string
label1 = 'HTE_WT'
label2 = 'HTE_KD'

master_rmats_validation(rmats_parsing_file,cuffdiff_file,df_name,bam1_list,bam2_list,label1,label2,exon_scale,intron_scale,sample_frac,sample_seed,flip_needed,fdr,dpsi,reads,one_variable_only,run_sashimi)
print('\n')
one_variable_only = 'yes'
master_rmats_validation(rmats_parsing_file,cuffdiff_file,df_name,bam1_list,bam2_list,label1,label2,exon_scale,intron_scale,sample_frac,sample_seed,flip_needed,fdr,dpsi,reads,one_variable_only,run_sashimi)


(47450, 'number_of_SE_events')
(198, 'number_of_SE_events_post_reads_fdr_dpsi')
(173, 'number_of_SE_events_post_FPKM')


(47450, 'number_of_SE_events')
(198, 'number_of_SE_events_post_reads_fdr_dpsi')
(173, 'number_of_SE_events_post_FPKM')
(106, 'number_of_SE_events_with_one_V_exon')
